In [2]:
import os
import numpy as np
import pandas as pd
import data_helpers
from data_helpers import TrainValTensorBoard
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Input, Embedding, Activation, Flatten, Dense, Concatenate
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model



In [3]:
#==================Preprocess===================

# Load data
csv = '../data/twitter/clean_tweet.csv'
df = pd.read_csv(csv, index_col=0)
print(df.head())

# Delete Null row
df = df.dropna()
print(df.target.value_counts())

# See word level length
length = [len(sent) for sent in df['text']]
print('The max length is: ', max(length))
print('The min length is: ', min(length))
print('The average length is: ', sum(length)/len(length))

/anaconda3/envs/py36/lib/python3.6/site-packages/numpy/lib/arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


                                                text  target
0  awww that s a bummer you shoulda got david car...       0
1  is upset that he can t update his facebook by ...       0
2  i dived many times for the ball managed to sav...       0
3     my whole body feels itchy and like its on fire       0
4  no it s not behaving at all i m mad why am i h...       0
0    798503
4    798250
Name: target, dtype: int64
The max length is:  137
The min length is:  1
The average length is:  62.53707430015788


In [22]:
df['target'] = df['target'].map({0: 0, 4: 1})

In [23]:
x_text = df['text'].values
y = df['target'].values

In [24]:
print(len(x_text))
print(x_text[:5])
print(len(y))
print(y[:5])
print(y[5:])

1596753
['awww that s a bummer you shoulda got david carr of third day to do it d'
 'is upset that he can t update his facebook by texting it and might cry as a result school today also blah'
 'i dived many times for the ball managed to save the rest go out of bounds'
 'my whole body feels itchy and like its on fire'
 'no it s not behaving at all i m mad why am i here because i can t see you all over there']
1596753
[0 0 0 0 0]
[0 0 0 ... 1 1 1]


In [25]:
# Positive sentence begin with 798503
np.where(y==1)

(array([ 798503,  798504,  798505, ..., 1596750, 1596751, 1596752]),)

In [12]:
# Tokenizer
tk = Tokenizer(num_words=None, oov_token='UNK')
tk.fit_on_texts(x_text)
# Convert string to index
sequences = tk.texts_to_sequences(x_text)

In [46]:
len(tk.word_index)

273721

In [27]:
print(sequences[:2])

[[447, 16, 12, 4, 1175, 8, 3453, 49, 833, 9238, 14, 1870, 33, 2, 42, 6, 136], [9, 781, 16, 84, 30, 13, 543, 183, 532, 123, 2003, 6, 7, 289, 527, 83, 4, 2308, 143, 41, 266, 1120]]


In [28]:
# Padding
sequences_pad = pad_sequences(sequences, maxlen=137, padding='post')

In [29]:
sequences_pad[:2]

array([[ 447,   16,   12,    4, 1175,    8, 3453,   49,  833, 9238,   14,
        1870,   33,    2,   42,    6,  136,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0],
       [   9,  781,   16,   84,   30,   13,  543,  183,  532,  123, 2003,

In [ ]:
# 10% for validation, 10% for test, and 80% for training
# all_indices = np.arange(len(y))
# validation_indices = int(all_indices*0.1)

In [39]:
# Shuffle data
np.random.seed(42)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = sequences_pad[shuffle_indices]
y_shuffled = y[shuffle_indices]

In [40]:
training_rate = 0.9
train_len = int(len(y) * training_rate)
x_train = x_shuffled[:train_len]
y_train = y_shuffled[:train_len]
x_test = x_shuffled[train_len:]
y_test = y_shuffled[train_len:]

In [45]:
print(len(y_train))
print(sum(y_train))
print()
print(len(y_test))
print(sum(y_test))

1437077
718479

159676
79771


In [38]:
train_len

1277402

In [47]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [48]:
from keras.utils import to_categorical
y = to_categorical(y)

In [49]:
y[5:]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [ ]:
#==================Preprocess===================

# Load data
csv = '../data/twitter/clean_tweet.csv'
df = pd.read_csv(csv, index_col=0)
print(df.head())

# Delete Null row
df = df.dropna()
print(df.target.value_counts())

# See word level length
length = [len(sent) for sent in df['text']]
print('The max length is: ', max(length))
print('The min length is: ', min(length))
print('The average length is: ', sum(length)/len(length))

df['target'] = df['target'].map({0: 0, 4: 1})

x_text = df['text'].values
y = df['target'].values

# Tokenizer
tk = Tokenizer(num_words=None, oov_token='UNK')
tk.fit_on_texts(x_text)
# Convert string to index
sequences = tk.texts_to_sequences(x_text)
# Padding
sequences_pad = pad_sequences(sequences, maxlen=137, padding='post')

# Shuffle data
np.random.seed(42)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = sequences_pad[shuffle_indices]
y_shuffled = y[shuffle_indices]

# Split train and test
training_rate = 0.9
train_len = int(len(y) * training_rate)
x_train = x_shuffled[:train_len]
y_train = y_shuffled[:train_len]
x_test = x_shuffled[train_len:]
y_test = y_shuffled[train_len:]
print('Training data size is: ', x_train.shape)
print('Validation data size is: ', x_test.shape)

